# 🧲 Magnetic System Explorer

Interactive simulation and visualization of magnetic systems with real-time monitoring.

**Features:**
- 🎯 Generic framework for any magnetic material
- 📊 Real-time thermodynamic property evolution
- 🎬 Animated spin configuration visualization
- ⚡ Efficient, compact code structure

## 🚀 Setup & Configuration

In [ ]:
# Essential imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display, clear_output
import time
from ase.build import bulk
import ipywidgets as widgets

# SpinLab imports
import spinlab
from spinlab import SpinSystem, MonteCarlo, ThermodynamicsAnalyzer
from spinlab.core.hamiltonian import Hamiltonian
from spinlab.core.fast_ops import check_numba_availability

# Configure plotting
plt.style.use('seaborn-v0_8' if 'seaborn-v0_8' in plt.style.available else 'default')
%matplotlib widget

# Check performance
numba_available, msg = check_numba_availability()
print(f"🔧 Numba: {msg}")
print(f"📦 SpinLab version: {spinlab.__version__ if hasattr(spinlab, '__version__') else 'dev'}")

## 🏗️ System Builder

In [ ]:
class MagneticSystemBuilder:
    """Efficient builder for magnetic systems."""
    
    @staticmethod
    def create_system(material='Fe', size=(12, 12, 1), exchange_J=-0.01, 
                      magnetic_field=None, anisotropy=None):
        """Create magnetic system with minimal code."""
        
        # Material database (expandable)
        materials = {
            'Fe': {'crystal': 'bcc', 'a': 2.87, 'neighbors': 3.5},
            'Ni': {'crystal': 'fcc', 'a': 3.52, 'neighbors': 3.8},
            'Co': {'crystal': 'hcp', 'a': 2.51, 'neighbors': 3.2},
            'Mn': {'crystal': 'bcc', 'a': 3.08, 'neighbors': 3.8}
        }
        
        mat = materials.get(material, materials['Fe'])
        
        # Build structure
        structure = bulk(material, mat['crystal'], a=mat['a'], cubic=True)
        structure = structure.repeat(size)
        
        # Build Hamiltonian
        hamiltonian = Hamiltonian()
        hamiltonian.add_exchange(J=exchange_J, neighbor_shell="shell_1")
        
        if magnetic_field:
            hamiltonian.add_magnetic_field(B_field=magnetic_field, g_factor=2.0)
        
        if anisotropy:
            hamiltonian.add_single_ion_anisotropy(A=anisotropy['K'], axis=anisotropy['axis'])
        
        # Create system
        system = SpinSystem(structure, hamiltonian, magnetic_model="3d")
        system.get_neighbors([mat['neighbors']])
        system.random_configuration()
        
        print(f"✅ {material} system: {len(structure)} sites ({size[0]}×{size[1]}×{size[2]})")
        return system

# Quick system creation
system = MagneticSystemBuilder.create_system(
    material='Fe', 
    size=(16, 16, 1),  # 2D-like system
    exchange_J=-0.02   # Strong ferromagnetic
)

## 📊 Real-Time Monitoring

In [ ]:
class RealTimeMonitor:
    """Compact real-time monitoring with live plots."""
    
    def __init__(self, figsize=(12, 4)):
        self.fig, (self.ax1, self.ax2, self.ax3) = plt.subplots(1, 3, figsize=figsize)
        self.data = {'steps': [], 'energy': [], 'magnetization': [], 'temperature': []}
        
        # Setup plots
        self.ax1.set_title('Energy Evolution')
        self.ax1.set_xlabel('MC Steps')
        self.ax1.set_ylabel('Energy (eV)')
        
        self.ax2.set_title('Magnetization')
        self.ax2.set_xlabel('MC Steps')
        self.ax2.set_ylabel('|M|')
        
        self.ax3.set_title('Spin Configuration')
        self.ax3.set_aspect('equal')
        self.ax3.axis('off')
        
        plt.tight_layout()
    
    def update(self, step, energy, magnetization, spins=None, temperature=None):
        """Update plots efficiently."""
        self.data['steps'].append(step)
        self.data['energy'].append(energy)
        self.data['magnetization'].append(np.linalg.norm(magnetization))
        if temperature:
            self.data['temperature'].append(temperature)
        
        # Update energy plot
        self.ax1.clear()
        self.ax1.plot(self.data['steps'], self.data['energy'], 'b-', alpha=0.7)
        self.ax1.set_title(f'Energy: {energy:.3f} eV')
        self.ax1.grid(True, alpha=0.3)
        
        # Update magnetization plot
        self.ax2.clear()
        self.ax2.plot(self.data['steps'], self.data['magnetization'], 'r-', alpha=0.7)
        self.ax2.set_title(f'|M|: {self.data["magnetization"][-1]:.3f}')
        self.ax2.grid(True, alpha=0.3)
        
        # Update spin configuration
        if spins is not None:
            self._plot_spins(spins)
        
        plt.draw()
    
    def _plot_spins(self, spins):
        """Efficient spin visualization."""
        self.ax3.clear()
        
        # Get 2D positions (assume z=0 for 2D-like systems)
        n_spins = len(spins)
        grid_size = int(np.sqrt(n_spins))
        
        if grid_size * grid_size == n_spins:  # Perfect square
            x = np.arange(grid_size)
            y = np.arange(grid_size)
            X, Y = np.meshgrid(x, y)
            
            # Spin directions and colors
            spins_2d = spins.reshape(grid_size, grid_size, 3)
            
            # Color by z-component
            colors = spins_2d[:, :, 2]
            
            # Plot as quiver (arrows)
            self.ax3.quiver(X, Y, spins_2d[:, :, 0], spins_2d[:, :, 1], 
                           colors, cmap='RdBu', scale=3, alpha=0.8)
            
            self.ax3.set_xlim(-1, grid_size)
            self.ax3.set_ylim(-1, grid_size)
            self.ax3.set_title(f'Spins ({grid_size}×{grid_size})')
            self.ax3.set_aspect('equal')

# Create monitor
monitor = RealTimeMonitor()
plt.show()

## 🎬 Interactive Simulation

In [ ]:
class InteractiveSimulation:
    """Compact simulation with live visualization."""
    
    def __init__(self, system, monitor):
        self.system = system
        self.monitor = monitor
        self.mc = None
    
    def run_simulation(self, temperature=100, n_steps=1000, update_interval=50):
        """Run simulation with real-time updates."""
        
        self.mc = MonteCarlo(self.system, temperature=temperature)
        
        print(f"🚀 Starting simulation: T={temperature}K, {n_steps} steps")
        
        start_time = time.time()
        
        for step in range(0, n_steps, update_interval):
            # Run batch of steps
            batch_steps = min(update_interval, n_steps - step)
            
            result = self.mc.run(
                n_steps=batch_steps, 
                equilibration_steps=0, 
                verbose=False
            )
            
            # Update visualization
            energy = result['final_energy']
            magnetization = result['final_magnetization']
            spins = self.system.spin_config
            
            self.monitor.update(step + batch_steps, energy, magnetization, spins, temperature)
            
            # Progress info
            elapsed = time.time() - start_time
            speed = (step + batch_steps) / elapsed if elapsed > 0 else 0
            
            clear_output(wait=True)
            print(f"Step {step + batch_steps}/{n_steps} | "
                  f"Speed: {speed:.1f} steps/s | "
                  f"Energy: {energy:.4f} eV | "
                  f"|M|: {np.linalg.norm(magnetization):.3f}")
            
            # Small delay for visualization
            time.sleep(0.1)
        
        total_time = time.time() - start_time
        print(f"\n✅ Simulation complete! Total time: {total_time:.2f}s")
        
        return result

# Create simulation
sim = InteractiveSimulation(system, monitor)

## 🎯 Quick Simulations

In [ ]:
# High temperature (paramagnetic)
result_high_T = sim.run_simulation(temperature=500, n_steps=800, update_interval=40)

In [ ]:
# Low temperature (ferromagnetic)
system.random_configuration()  # Reset
result_low_T = sim.run_simulation(temperature=50, n_steps=800, update_interval=40)

## 🌡️ Temperature Sweep

In [ ]:
def temperature_sweep(system, temperatures, n_steps_per_T=200):
    """Efficient temperature sweep with live plotting."""
    
    results = {'T': [], 'energy': [], 'magnetization': [], 'heat_capacity': []}
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
    
    for i, T in enumerate(temperatures):
        print(f"🌡️  T = {T:.1f}K ({i+1}/{len(temperatures)})")
        
        mc = MonteCarlo(system, temperature=T)
        result = mc.run(n_steps=n_steps_per_T, equilibration_steps=50, verbose=False)
        
        # Store results
        results['T'].append(T)
        results['energy'].append(result['final_energy'] / len(system.positions))
        results['magnetization'].append(np.linalg.norm(result['final_magnetization']))
        
        # Update plots
        ax1.clear()
        ax1.plot(results['T'], results['energy'], 'bo-', markersize=4)
        ax1.set_xlabel('Temperature (K)')
        ax1.set_ylabel('Energy per site (eV)')
        ax1.set_title('Energy vs Temperature')
        ax1.grid(True, alpha=0.3)
        
        ax2.clear()
        ax2.plot(results['T'], results['magnetization'], 'ro-', markersize=4)
        ax2.set_xlabel('Temperature (K)')
        ax2.set_ylabel('|Magnetization|')
        ax2.set_title('Magnetization vs Temperature')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.draw()
        plt.pause(0.1)
    
    print("✅ Temperature sweep complete!")
    return results

# Run temperature sweep
temperatures = np.linspace(10, 400, 15)
sweep_results = temperature_sweep(system, temperatures)
plt.show()

## 🎮 Interactive Controls

In [ ]:
# Interactive widgets for live control
def create_interactive_controls():
    """Create interactive simulation controls."""
    
    # Widgets
    temp_slider = widgets.FloatSlider(
        value=100, min=10, max=500, step=10,
        description='Temperature:', style={'description_width': 'initial'}
    )
    
    steps_slider = widgets.IntSlider(
        value=500, min=100, max=2000, step=100,
        description='MC Steps:', style={'description_width': 'initial'}
    )
    
    material_dropdown = widgets.Dropdown(
        options=['Fe', 'Ni', 'Co', 'Mn'],
        value='Fe',
        description='Material:'
    )
    
    run_button = widgets.Button(
        description='🚀 Run Simulation',
        button_style='success'
    )
    
    reset_button = widgets.Button(
        description='🔄 Reset System',
        button_style='warning'
    )
    
    output = widgets.Output()
    
    def on_run_clicked(b):
        with output:
            clear_output(wait=True)
            # Create new system if material changed
            global system, sim
            system = MagneticSystemBuilder.create_system(
                material=material_dropdown.value,
                size=(12, 12, 1)
            )
            sim = InteractiveSimulation(system, monitor)
            
            # Run simulation
            sim.run_simulation(
                temperature=temp_slider.value,
                n_steps=steps_slider.value,
                update_interval=50
            )
    
    def on_reset_clicked(b):
        with output:
            clear_output(wait=True)
            system.random_configuration()
            print("🔄 System reset to random configuration")
    
    run_button.on_click(on_run_clicked)
    reset_button.on_click(on_reset_clicked)
    
    # Layout
    controls = widgets.VBox([
        widgets.HBox([material_dropdown, temp_slider]),
        widgets.HBox([steps_slider]),
        widgets.HBox([run_button, reset_button]),
        output
    ])
    
    return controls

# Display controls
controls = create_interactive_controls()
display(controls)

## 📈 Quick Analysis

In [ ]:
# Quick thermodynamic analysis
def quick_analysis(sweep_results):
    """Extract key thermodynamic insights."""
    
    T = np.array(sweep_results['T'])
    E = np.array(sweep_results['energy'])
    M = np.array(sweep_results['magnetization'])
    
    # Heat capacity (numerical derivative)
    if len(T) > 2:
        C = -np.gradient(E, T) * T
        
        # Find critical temperature (peak in heat capacity)
        T_c_idx = np.argmax(C)
        T_c = T[T_c_idx]
        
        print(f"📊 Quick Analysis:")
        print(f"   Critical Temperature: ~{T_c:.1f}K")
        print(f"   Low-T magnetization: {M[0]:.3f}")
        print(f"   High-T magnetization: {M[-1]:.3f}")
        print(f"   Energy range: {E.min():.4f} to {E.max():.4f} eV/site")
        
        # Simple phase diagram
        if M[0] > 0.5 and M[-1] < 0.2:
            print(f"   Phase transition: Ferromagnetic → Paramagnetic")
        
        return {'T_c': T_c, 'heat_capacity': C}
    
    return None

if 'sweep_results' in locals():
    analysis = quick_analysis(sweep_results)

## 💡 Usage Tips

**Efficient Workflow:**
1. 🏗️ **System Builder**: Create any magnetic material quickly
2. 🎬 **Interactive Simulation**: Watch evolution in real-time
3. 🌡️ **Temperature Sweep**: Find phase transitions
4. 🎮 **Interactive Controls**: Experiment with parameters

**Performance:**
- Uses Numba acceleration for speed
- Efficient visualization updates
- Compact code with maximum functionality

**Customization:**
- Add new materials to `MagneticSystemBuilder`
- Modify Hamiltonian parameters
- Adjust visualization styles